In [ ]:
pip uninstall -y transformers peft

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2


In [ ]:
pip install transformers==4.37.2 peft==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install accelerate==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.8.1
    Uninstalling accelerate-1.8.1:
      Successfully uninstalled accelerate-1.8.1


In [ ]:
!pip install -U datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [ ]:
import transformers
print(transformers.__version__)
print(transformers.__file__)

4.37.2
/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


## 🔹 Imports and Hyperparameter Grid

In [ ]:
from torch.utils.data import Dataset
from datetime import datetime
import gc
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support, roc_auc_score,
                             confusion_matrix, ConfusionMatrixDisplay, classification_report,
                             roc_curve, auc, f1_score)
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys
import torch
import torch.nn.functional as F
import torch.nn as nn
from transformers import (AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments,
                          Trainer, EarlyStoppingCallback, set_seed)

# Define hyperparameter grid
hyperparameter_grid = [
    {"learning_rate": 5e-5, "weight_decay": 0.01},
    {"learning_rate": 3e-5, "weight_decay": 0.01},
    {"learning_rate": 2e-5, "weight_decay": 0.1},
    {"learning_rate": 2e-5, "weight_decay": 0.01},  # baseline
    {"learning_rate": 1e-5, "weight_decay": 0.01},
]

## 🔹 Utility Functions

In [ ]:
def tokenize_and_cache(emails, tokenizer_name="microsoft/deberta-v3-base", max_length=320,
                       save_path="tokenized_emails.pt", force_retokenize=False):
    if Path(save_path).exists() and not force_retokenize:
        print(f"Loading tokenized data from '{save_path}'...")
        return torch.load(save_path)
    print(f"Tokenizing {len(emails)} emails with max_length={max_length}...")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenized = tokenizer(
        emails, max_length=max_length, padding="max_length", truncation=True,
        return_tensors="pt"
    )
    torch.save(tokenized, save_path)
    print(f"Saved tokenized data to '{save_path}'")
    return tokenized

def save_classification_report(report_str, filename):
    with open(filename, 'w') as f:
        f.write(report_str)
    print(f"Saved classification report to {filename}")

def compute_metrics(eval_pred):
    # eval_pred is now a `EvalPrediction` object with .predictions and .label_ids
    logits = eval_pred.predictions[0]
    labels = eval_pred.label_ids

    # Sanitize logits
    logits = np.array(logits, dtype=np.float32)
    if not np.isfinite(logits).all():
        print("Detected NaN or Inf in logits before softmax")
        logits = np.nan_to_num(logits, nan=0.0, posinf=10.0, neginf=-10.0)

    predictions = np.argmax(logits, axis=-1)

    # Convert logits to probabilities using softmax
    probs = softmax(logits, axis=1)[:, 1] if logits.shape[1] > 1 else softmax(logits, axis=1)[:, 0]

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary', zero_division=0
    )
    acc = accuracy_score(labels, predictions)

    try:
        roc_auc = roc_auc_score(labels, probs)
    except ValueError:
        roc_auc = float('nan')

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "roc_auc": roc_auc,
    }

In [ ]:
# Dataset class
class EmailDataset(Dataset):
    def __init__(self, df):
        self.encodings = tokenizer(list(df["cleaned text"]), truncation=True, padding="max_length", max_length=512)
        self.features = df[["link_count", "email length"]].values
        assert np.isfinite(self.features).all(), "Detected NaN or Inf in input features"
        self.labels = df["label"].values.astype(int)
        assert not np.isnan(self.labels).any(), "NaNs in labels"

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["features"] = torch.tensor(self.features[idx], dtype=torch.float32)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
# Model class
class DebertaWithFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        self.deberta = AutoModel.from_pretrained("microsoft/deberta-v3-base")
        self.config = self.deberta.config
        self.config.return_dict = True
        self.hidden_size = self.deberta.config.hidden_size
        self.feature_proj = nn.Linear(2, 32)
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size + 32, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )

    def forward(self, input_ids, attention_mask, features, labels=None):
      deberta_output = self.deberta(input_ids=input_ids, attention_mask=attention_mask)

      if self.config.return_dict:
          cls_output = deberta_output.last_hidden_state[:, 0]
      else:
          cls_output = deberta_output[0][:, 0]  # first element of tuple, first token CLS

      feature_proj = self.feature_proj(features)
      feature_proj = torch.clamp(feature_proj, -10.0, 10.0)
      combined = torch.cat((cls_output, feature_proj), dim=1)
      logits = self.classifier(combined)

      if torch.isnan(logits).any() or torch.isinf(logits).any():
        print("NaNs in logits at batch")
        print("CLS shape:", cls_output.shape)
        print("Features shape:", features.shape)
        print("Feature values:", features[0])

      loss = None
      if labels is not None:
          loss_fn = nn.CrossEntropyLoss()
          loss = loss_fn(logits, labels)

      return {"loss": loss, "logits": logits, "labels": labels}

In [ ]:
def collate_fn(batch):
        return {
            "input_ids": torch.stack([x["input_ids"] for x in batch]),
            "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
            "features": torch.stack([x["features"] for x in batch]),
            "labels": torch.tensor([int(x["labels"]) for x in batch], dtype=torch.long)
        }

## 🔹 Load Dataset and Setup

In [ ]:
experiment_index = 4  # replace with desired config index
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define a base path in Google Drive to save outputs
data_path = '/content/drive/My Drive/Cybersecurity Practicum/'
drive_base_path = '/content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results'
Deberta_data_path = '/content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_data'
report_path = '/content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results'
report_dir = os.path.join(report_path, f"config_{experiment_index}")
import os
os.makedirs(drive_base_path, exist_ok=True)
os.makedirs(Deberta_data_path, exist_ok=True)
os.makedirs(report_dir, exist_ok=True)

Mounted at /content/drive


In [ ]:
config = hyperparameter_grid[experiment_index]
print(f"Running experiment with config: {config}")

df = pd.read_csv(os.path.join(data_path, "finalized_dataset.csv"))
df["cleaned text"] = df["cleaned text"].astype(str)
# Ensure label column is integer type
df["label"] = df["label"].astype(int)

print("Missing in features:\n", df[["link_count", "email length"]].isna().sum())
print("Feature value stats:\n", df[["link_count", "email length"]].describe())

# Fix extreme outliers
df["link_count"] = df["link_count"].clip(0, 100)
df["email length"] = df["email length"].clip(0, 20000)

# Split into training (80%) and test (20%) stratified
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# Normalize features
scaler = StandardScaler()
train_df[["link_count", "email length"]] = scaler.fit_transform(train_df[["link_count", "email length"]])
test_df[["link_count", "email length"]] = scaler.transform(test_df[["link_count", "email length"]])

# Save full train/test split
train_df.to_csv(Deberta_data_path + "/train_full.csv", index=False)
test_df.to_csv(Deberta_data_path + "/test_holdout.csv", index=False)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# Stratified K-Fold with HuggingFace Trainer
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train_df.index.values
y = train_df["label"].values

best_f1 = 0.0
best_model_dir = None

all_fold_metrics = []

Running experiment with config: {'learning_rate': 1e-05, 'weight_decay': 0.01}
Missing in features:
 link_count      0
email length    0
dtype: int64
Feature value stats:
          link_count  email length
count  87792.000000  8.779200e+04
mean       3.101911  1.239132e+03
std       10.299092  1.498235e+04
min        0.000000  9.000000e+00
25%        0.000000  2.770000e+02
50%        1.000000  5.370000e+02
75%        2.000000  1.272250e+03
max     1029.000000  4.279447e+06


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## 🔹 Cross-Validation Training Loop

In [ ]:
k = 5
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n===== Fold {fold}/{k} =====")

        fold_train_df = df.iloc[train_idx].reset_index(drop=True)
        os.makedirs(f"{Deberta_data_path}/fold_{fold}/", exist_ok=True)
        os.makedirs(f"{Deberta_data_path}/fold_{fold}/", exist_ok=True)
        fold_train_df.to_csv(f"{Deberta_data_path}/fold_{fold}/train.csv", index=False)
        fold_val_df = df.iloc[val_idx].reset_index(drop=True)
        fold_val_df.to_csv(f"{Deberta_data_path}/fold_{fold}/val.csv", index=False)

        # Sanity checks
        assert fold_val_df["label"].notna().all(), f"NaNs in validation labels for fold {fold}"
        assert fold_train_df["label"].notna().all(), f"NaNs in training labels for fold {fold}"
        assert set(fold_val_df["label"].unique()).issubset({0, 1}), f"Unexpected labels in validation set for fold {fold}"
        assert set(fold_train_df["label"].unique()).issubset({0, 1}), f"Unexpected labels in training set for fold {fold}"

        if len(np.unique(fold_val_df["label"])) < 2:
          print(f"Warning: Fold {fold} validation set has only one class!")

        train_dataset = EmailDataset(fold_train_df)
        val_dataset = EmailDataset(fold_val_df)

        set_seed(42)
        model = DebertaWithFeatures()

        fold_output_dir = os.path.join(report_dir, f"fold{fold}_results")
        if os.path.exists(fold_output_dir):
          print(f"Fold {fold} already exists, skipping...")
          continue

        training_args = TrainingArguments(
            output_dir=fold_output_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=config["learning_rate"],
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=3,
            weight_decay=config["weight_decay"],
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            logging_dir=os.path.join(report_dir, "logs"),
            logging_steps=10,
            fp16=False,
            max_grad_norm=1.0,  # Clipping to prevent exploding gradients
            gradient_accumulation_steps=1,
            overwrite_output_dir=True
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
            data_collator=collate_fn
        )

        trainer.model.config.return_dict = True

        print(f"Training Model on fold {fold}")
        trainer.train()

        print(f"Obtaining fold metrics for fold {fold}")
        fold_metrics_over_epochs = []
        for entry in trainer.state.log_history:
            if "eval_f1" in entry and "epoch" in entry:
                fold_metrics_over_epochs.append({
                    "fold": fold,
                    "epoch": entry["epoch"],
                    "f1": entry["eval_f1"],
                    "accuracy": entry.get("eval_accuracy"),
                    "precision": entry.get("eval_precision"),
                    "recall": entry.get("eval_recall"),
                })
        all_fold_metrics.extend(fold_metrics_over_epochs)

        # Extract optimizer info
        optimizer_type = type(trainer.optimizer).__name__
        optimizer_params = trainer.optimizer.param_groups[0]

        learning_rate = optimizer_params.get("lr", "N/A")
        weight_decay = training_args.weight_decay
        betas = optimizer_params.get("betas", ("N/A", "N/A"))
        epsilon = training_args.adam_epsilon if hasattr(training_args, "adam_epsilon") else "N/A"

        # DeBERTa uses CrossEntropyLoss for classification by default
        loss_type = "CrossEntropyLoss"

        # Classification report & confusion matrix for validation set
        val_preds = trainer.predict(val_dataset)

        # Save misclassified samples
        misclassified = fold_val_df.copy()
        logits = val_preds.predictions[0]
        preds = np.argmax(logits, axis=-1)
        probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1).numpy()
        misclassified["predicted_label"] = preds
        misclassified["confidence"] = probs[np.arange(len(preds)), preds]  # probability of predicted class
        misclassified = misclassified[misclassified["label"] != misclassified["predicted_label"]]
        os.makedirs(f"{report_dir}/fold_{fold}", exist_ok=True)
        misclassified.to_csv(f"{report_dir}/fold_{fold}/misclassified.csv", index=False)

        y_val_true = val_preds.label_ids
        f1 = f1_score(y_val_true, preds)
        print(f"Fold {fold} F1 Score: {f1:.4f}")

        if f1 > best_f1:
          best_f1 = f1
          best_model_dir = os.path.join(report_dir, f"best_model_fold_{fold}")
          os.makedirs(best_model_dir, exist_ok=True)
          torch.save(model.state_dict(), os.path.join(best_model_dir, "model.pt"))
          tokenizer.save_pretrained(best_model_dir)
          joblib.dump(scaler, os.path.join(best_model_dir, "scaler.pkl"))
          print(f"New best model saved to {best_model_dir}")

        val_class_report = classification_report(
            y_val_true, preds, target_names=["Legit", "Phish"], digits=4
        )
        # Full report
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        report_text = f"""
        ==================== EVALUATION REPORT ====================

        Timestamp       : {timestamp}
        Model           : microsoft/deberta-v3-base
        Tokenizer       : microsoft/deberta-v3-base
        Token Length    : 512
        Max Epochs      : {training_args.num_train_epochs}
        Best Metric     : {training_args.metric_for_best_model}

        ---------------- Optimizer & Loss Info ----------------
        Optimizer       : {optimizer_type}
        Learning Rate   : {learning_rate}
        Weight Decay    : {weight_decay}
        Betas           : {betas}
        Epsilon         : {epsilon}
        Loss Function   : {loss_type}
        ---------------- Classification Report ----------------
        {val_class_report}
        """
        print(f"\nClassification report for Fold {fold} Validation:\n{val_class_report}")

        report_path = os.path.join(report_dir, f"fold{fold}_val_classification_report.txt")
        save_classification_report(report_text, report_path)

        cm_val = confusion_matrix(y_val_true, preds)
        disp_val = ConfusionMatrixDisplay(cm_val, display_labels=["Legit", "Phish"])
        disp_val.plot(cmap='Blues')
        plt.title(f"Confusion Matrix - Validation Fold {fold}")
        plt.savefig(os.path.join(report_dir, f"fold{fold}_val_confusion_matrix.png"))
        plt.close()

        # Explicitly delete model, datasets, and predictions
        del model
        del train_dataset
        del val_dataset
        del preds
        del probs

        # Force garbage collection and clear CUDA cache
        import gc
        gc.collect()
        torch.cuda.empty_cache()


===== Fold 1/5 =====


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Fold 1 already exists, skipping...

===== Fold 2/5 =====


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Fold 2 already exists, skipping...

===== Fold 3/5 =====


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jessica-borowy-1 (jessica-borowy-1-georgia-tech-yellow-jackets) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.011300,0.040024,0.991029,0.991888,0.997281,0.986552,0.999653
2,0.095000,0.029428,0.994589,0.995147,0.992233,0.998079,0.999729
3,0.000100,0.024374,0.995586,0.996028,0.996538,0.995517,0.999789


Obtaining fold metrics for fold 5


Fold 5 F1 Score: 0.9960
New best model saved to /content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results/config_4/best_model_fold_5

Classification report for Fold 5 Validation:
              precision    recall  f1-score   support

       Legit     0.9944    0.9957    0.9950      6238
       Phish     0.9965    0.9955    0.9960      7808

    accuracy                         0.9956     14046
   macro avg     0.9955    0.9956    0.9955     14046
weighted avg     0.9956    0.9956    0.9956     14046

Saved classification report to /content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results/config_4/fold5_val_classification_report.txt


In [ ]:
# Aggregate and save fold metrics over time
metrics_df = pd.DataFrame(all_fold_metrics)
fold_metrics_path = os.path.join(report_dir, "fold_metrics_over_time.csv")
metrics_df.to_csv(fold_metrics_path, index=False)
print(f"Fold metrics over time saved to {fold_metrics_path}")

Fold metrics over time saved to /content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results/config_4/fold_metrics_over_time.csv


In [ ]:
# Plot F1 and other metrics over epochs per fold
# Use a high-contrast palette
palette = sns.color_palette("colorblind", n_colors=metrics_df['fold'].nunique())
plt.figure(figsize=(12, 6))
sns.lineplot(data=metrics_df, x="epoch", y="f1", hue="fold", palette=palette, marker="o")
plt.title("F1 Score over Epochs per Fold")
plt.xlabel("Epoch")
plt.ylabel("F1 Score")
plt.grid(True)
plt.legend(title="Fold")
plt.tight_layout()
plt.savefig(os.path.join(report_dir, "f1_over_time_across_folds.png"))
plt.close()

In [ ]:
for metric in ["accuracy", "precision", "recall"]:
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=metrics_df, x="epoch", y=metric, hue="fold", palette=palette, marker="o")
        plt.title(f"{metric.capitalize()} over Epochs per Fold")
        plt.xlabel("Epoch")
        plt.ylabel(metric.capitalize())
        plt.grid(True)
        plt.legend(title="Fold")
        plt.tight_layout()
        plt.savefig(os.path.join(report_dir, f"{metric}_over_time_across_folds.png"))
        plt.close()

In [ ]:
best_row = metrics_df.loc[metrics_df['f1'].idxmax()]
best_fold = int(best_row['fold'])
best_epoch = int(best_row['epoch'])

print(f"\nBest model: Fold {best_fold} at epoch {best_epoch} with F1={best_row['f1']:.4f}")


Best model: Fold 5 at epoch 3 with F1=0.9960


In [ ]:
# Final Test Evaluation
print("\nEvaluating on held-out test set using best model")


Evaluating on held-out test set using best model


In [ ]:
# Load best model
set_seed(42)
best_model = DebertaWithFeatures()
best_model_dir = os.path.join(report_dir, f"best_model_fold_4")
print(f"Loading best model from {best_model_dir}")
best_model.load_state_dict(torch.load(os.path.join(best_model_dir, "model.pt")))
best_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Loading best model from /content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results/config_4/best_model_fold_4


DebertaWithFeatures(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        

In [ ]:
# Tokenizer already loaded
scaler = joblib.load(os.path.join(best_model_dir, "scaler.pkl"))

## 🔹 Final Evaluation on Test Set

In [ ]:
# Prepare test dataset
test_dataset = EmailDataset(test_df)

In [ ]:
# Evaluate best model on test set
test_training_args = TrainingArguments(
    output_dir=os.path.join(report_dir, "test_results"),
    per_device_eval_batch_size=64,
    do_train=False,
    do_eval=True,
    fp16=True,
)
test_trainer = Trainer(
    model=best_model,
    args=test_training_args,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
test_metrics = test_trainer.evaluate(test_dataset)
print("\nTest set evaluation metrics:")
print(test_metrics)

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jessica-borowy-1 (jessica-borowy-1-georgia-tech-yellow-jackets) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Test set evaluation metrics:
{'eval_loss': 0.00928623415529728, 'eval_accuracy': 0.9977219659433908, 'eval_f1': 0.9979253112033195, 'eval_precision': 0.9983395599833956, 'eval_recall': 0.9975114060555786, 'eval_roc_auc': 0.9999236430311379, 'eval_runtime': 69.0401, 'eval_samples_per_second': 254.33, 'eval_steps_per_second': 3.983}


In [ ]:
test_preds = test_trainer.predict(test_dataset)

# Get full logits
logits = test_preds.predictions[0]  # shape: (num_samples, num_classes)
y_test_pred = np.argmax(logits, axis=-1)
y_test_true = test_preds.label_ids

In [ ]:
# ROC Curve - Use probabilities for the positive class (class 1)
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
fpr, tpr, _ = roc_curve(y_test_true, probs)
roc_auc = auc(fpr, tpr)

# Save ROC curve plot
roc_path = os.path.join(report_dir, "roc_curve_test_set.png")
plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Test Set")
plt.legend(loc="lower right")
plt.savefig(roc_path)
plt.close()

print(f"ROC Curve graph saved at {roc_path}")

ROC Curve graph saved at /content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results/config_4/roc_curve_test_set.png


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test_true, y_test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Legit", "Phish"])
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - Phishing Detection")
plt.tight_layout()
cm_path1 = os.path.join(report_dir, "test_confusion_matrix.png")
plt.savefig(cm_path1)
plt.close()

In [ ]:
# Confusion matrix (normalized)
cm = confusion_matrix(y_test_true, y_test_pred, normalize='true')  # normalize by true labels (rows)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Legit", "Phish"])
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Normalized Confusion Matrix - Phishing Detection")
plt.tight_layout()
cm_path2 = os.path.join(report_dir, "test_confusion_matrix_normalized.png")
plt.savefig(cm_path2)
plt.close()

In [ ]:
test_class_report = classification_report(
    y_test_true, y_test_pred, target_names=["Legit", "Phish"], digits=4
)

In [ ]:
# Full report
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
report_text = f"""
==================== EVALUATION REPORT ====================

Timestamp       : {timestamp}
Model           : microsoft/deberta-v3-base
Tokenizer       : microsoft/deberta-v3-base
Token Length    : 320
Max Epochs      : {training_args.num_train_epochs}
Best Metric     : {training_args.metric_for_best_model}

---------------- Optimizer & Loss Info ----------------
Optimizer       : {optimizer_type}
Learning Rate   : {learning_rate}
Weight Decay    : {weight_decay}
Betas           : {betas}
Epsilon         : {epsilon}
Loss Function   : {loss_type}

---------------- Evaluation Metrics ----------------
{json.dumps(test_metrics, indent=4)}

ROC Curve visualization saved at: {roc_path}

---------------- Classification Report ----------------
{test_class_report}

Confusion Matrix Saved at: {cm_path1}
Normalized Confusion Matrix Saved at: {cm_path2}

============================================================
"""

In [ ]:
print("\nClassification report for Test Set:\n", test_class_report)
test_report_path = os.path.join(report_dir, "test_set_classification_report.txt")
save_classification_report(report_text, test_report_path)


Classification report for Test Set:
               precision    recall  f1-score   support

       Legit     0.9970    0.9980    0.9975      7915
       Phish     0.9983    0.9975    0.9979      9644

    accuracy                         0.9977     17559
   macro avg     0.9977    0.9977    0.9977     17559
weighted avg     0.9977    0.9977    0.9977     17559

Saved classification report to /content/drive/My Drive/Cybersecurity Practicum/Custom_CV_phishing_results/New_Deberta_results/config_4/test_set_classification_report.txt
